In [1]:
import csv
from collections import namedtuple
from contextlib import contextmanager
import re

## Part-1: Context Manager + Generator - Using Custom Class

In [2]:
class CSVReader:
    ''' Reads the CSV file, generates a "Generator" where data is parses in named tuple
    '''
    def __init__(self, filename: str, mode: str='r', separator: str=','):
        ''' Initializes the class with properties required for reading the file
        
        Parameters:
        ----------
        file_name:  File Name in STRING, which to be read
        mode:       'r' for read, 'w' for write and 'a' for append. Although only 'r' is supported
        separator:  Seperator which seperates the data in each row. Generally it will be ',' 
                    but it can be any charecter
        
        '''
        if mode not in ['r']:
            raise ValueError(f"Mode {mode} is not supported. Only 'r' (read) mode is allowed.")
        
        self._filename = filename
        self._mode = mode
        self._separator = separator
        # Header of the CSV file is saved in this list. It is assumed that only one 
        # line of header is present in the CSV file    
        self._fd = None
        self._header = []
        self._row_data = None

    def __enter__(self):
        ''' When enter the context, read the first line (which is header) and return the object
        '''
        try:
            self._fd = open(self._filename, self._mode, newline='', encoding='utf-8')
            self._csv_reader = csv.reader(self._fd, delimiter=self._separator)
            self._header = next(self._csv_reader)  # Read the header row
            
            # Sanitize the headers to be valid Python identifiers
            self._header = [re.sub(r'\W|^(?=\d)', '_', h) for h in self._header]
            
            self._row_data = namedtuple('row_data', self._header)
            return self

        except IOError as e:
            print(f"Error opening file: {e}")
            raise        

    def __iter__(self):
        ''' Iterator for the loop 
        '''
        return self

    def __next__(self):
        ''' To fetch the next time. Each row is fetched and then new line charecter is stripped
        and each row is then split according to "seperator" which was sending during construction 
        of the object 
        '''
        # Return the next row as a named tuple
        try:
            row = next(self._csv_reader)
            return self._row_data(*row)
        except StopIteration:
            raise StopIteration

    def __exit__(self, exc_type, exc_value, traceback):
        ''' Upon exiting the context close the file and display any error/exception occurred. 
        if exception has occured then stop the excecution of the code
        '''
        self._fd.close()
        if exc_type:
            print(f"Exception {exc_type} occurred: {exc_value}")
            
        if self._fd.closed:
            print("File closed properly")
        else:
            print("File did not close properly.")
        return False

### Open person_info.csv and display first 10 rows

In [3]:
# Read CSV and display rows
display_row_count = 0
with CSVReader('personal_info.csv', mode='r', separator=',') as csv_reader:
    print(csv_reader._header)
    for row in csv_reader:
        print(row)
        display_row_count += 1
        if (display_row_count > 10):
            break

['ssn', 'first_name', 'last_name', 'gender', 'language']
row_data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
row_data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
row_data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
row_data(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
row_data(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')
row_data(ssn='105-27-5541', first_name='Federico', last_name='Aggett', gender='Male', language='Chinese')
row_data(ssn='105-85-7486', first_name='Angelina', last_name='McAvey', gender='Female', language='Punjabi')
row_data(ssn='105-91-5022', first_name='Moselle', last_name='Apfel', gender='Female', language='Latvian')
row_data(ssn='105-91-7777', first_name='Audi', last_name='Roach', gender='Female',

### Open cars-2.csv and display first 10 rows

In [4]:
# Read CSV and display rows
display_row_count = 0
with CSVReader('cars-2.csv', mode='r', separator=';') as csv_reader:
    print(csv_reader._header)
    for row in csv_reader:
        print(row)
        display_row_count += 1
        if (display_row_count > 10):
            break

['Car', 'MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model', 'Origin']
row_data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
row_data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
row_data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
row_data(Car='AMC Rebel SST', MPG='16.0', Cylinders='8', Displacement='304.0', Horsepower='150.0', Weight='3433.', Acceleration='12.0', Model='70', Origin='US')
row_data(Car='Ford Torino', MPG='17.0', Cylinders='8', Displacement='302.0', Horsepower='140.0', Weight='3449.', Acceleration='10.5', Model='70', Origin='US')
row_data(Car='Ford Galaxie 500', MPG='15.0', Cylinders='8', Displacement=

## Part-2: Context Manager + Generator - Using Built-in contextmanager Decorator

In [5]:
@contextmanager
def csv_reader_generator(filename :str, mode :str = 'r', separator :str= ','):
    ''' Function helps in creating the "Generator" by reading a csv file
    '''
    print(f"Opening the file {filename} in {mode} mode...")

    try:
        fd = open(filename, mode)
        csv_reader = csv.reader(fd, delimiter=separator)
        header = next(csv_reader)  # Read the header row
        
        # Sanitize the header to be valid Python identifiers
        header = [re.sub(r'\W|^(?=\d)', '_', h) for h in header]
        print("Header:", header)        

        # Yielding a generator (lazy iterator)
        yield (csv_reader)
    except FileNotFoundError:
        print("No such file exists in this path")
    finally:
        fd.close()

In [6]:
# Read CSV and display rows
display_row_count = 0
with csv_reader_generator('personal_info.csv', mode='r', separator=',') as rows:
    for row in rows:
        print(row)
        display_row_count += 1
        if (display_row_count > 10):
            break

Opening the file personal_info.csv in r mode...
Header: ['ssn', 'first_name', 'last_name', 'gender', 'language']
['100-53-9824', 'Sebastiano', 'Tester', 'Male', 'Icelandic']
['101-71-4702', 'Cayla', 'MacDonagh', 'Female', 'Lao']
['101-84-0356', 'Nomi', 'Lipprose', 'Female', 'Yiddish']
['104-22-0928', 'Justinian', 'Kunzelmann', 'Male', 'Dhivehi']
['104-84-7144', 'Claudianus', 'Brixey', 'Male', 'Afrikaans']
['105-27-5541', 'Federico', 'Aggett', 'Male', 'Chinese']
['105-85-7486', 'Angelina', 'McAvey', 'Female', 'Punjabi']
['105-91-5022', 'Moselle', 'Apfel', 'Female', 'Latvian']
['105-91-7777', 'Audi', 'Roach', 'Female', 'Estonian']
['106-35-1938', 'Mackenzie', 'Nussey', 'Male', 'Swedish']
['106-36-3293', 'Martino', 'Tregoning', 'Male', 'Tok Pisin']


In [7]:
# Read CSV and display rows
display_row_count = 0
with csv_reader_generator('cars-2.csv', mode='r', separator=';') as rows:
    for row in rows:
        print(row)
        display_row_count += 1
        if (display_row_count > 10):
            break

Opening the file cars-2.csv in r mode...
Header: ['Car', 'MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model', 'Origin']
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']
['Chevrolet Impala', '14.0', '8', '454.0', '220.0', '4354.', '9.0', '70', 'US']
['Plymouth Fury iii', '14.0', '8', '440.0', '215.0', '4312.', '8.5', '70', 'US']
['Pontiac Catalina', '14.0', '8', '455.0', '225.0', '4425.', '10.0', '70', 'US']
['AMC Ambassador DPL', '15.0', '8', '390.0', '190.0', '3850.', '8.5', '70', 'US']
['Citroen DS-21 Pallas', '0', '4',